<a href="https://colab.research.google.com/github/kameda-yoshinari/DataAlgo-UT/blob/main/DataAlgo_UT(017)_FPTAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 8.2. FPTAS

本節では，近似精度と計算量との関係についてさらに学習する．




**いつもの約束**  
１つのコードセルだけの実行は Ctrl + Enter．  
エディタで「インデント幅（スペース）は4で表示」「行番号を表示」「インデントガイドを表示」．  
内部では日本語はUTF-8で表現されている．


# 準備

インスタンスに接続し起動する．  
下記の手順でGoogle Driveをマウントする．  
マウント先に移動し，作業フォルダとする．  
これによって，インスタンスがリセットされてもGoogle Drive内にファイルが保存されるようにする．

In [ ]:
!echo "Google Driveをマウントします"
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!echo "今回の作業用フォルダを作成しそこに移動します"
%cd /content/drive/My\ Drive/
%mkdir -p UT_DataAlgo/DA_017
%cd       UT_DataAlgo/DA_017
!ls
!echo "日本時間表示"
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Japan /etc/localtime
!date

# FPTAS

近似解を多項式時間で得る近似アルゴリズムはいつも発明発見できるわけではない．  
実際，多くの研究努力がこの近似アルゴリズムの発明発見に費やされている．  

今，最小値問題において，α近似アルゴリズムが見つかったとする．(α>=1)

8.1.節の例ではα=2と固定値であったが，これでは使い勝手がよくない．

そこで，α=1+ε (ε>0) とし，多項式時間で任意のεについて解を求めることができる近似アルゴリズムを，特に Polynomial Time Approximation Scheme (PTAS) と呼ぶ．

これは大いにありがたいアルゴリズムである．

もちろんそんなにうまい話がいつもあるはずがなく，多くの場合，ε→0にしていくと，その小ささ（多くの場合は 1/ε）に対して指数的に計算時間が必要となってしまうことが多い．

ところが，ごく稀にこの 1/ε に対して計算時間の増加が1/εの多項式時間でしか増加しないアルゴリズムが発明発見されることがある．この近似アルゴリズムのことを，特に敬意を込めて，Fully Polynomial Time Approximation Scheme (FPTAS) と呼ぶ．








# ナップサック問題におけるFPTASアルゴリズム

6.3.節で取り上げたn個に対する0-1整数ナップサック問題について再び取り上げる．

ナップサック問題は最大化問題なので，任意のε(ε>0)について，最適解に対して近似解を 1-ε の近似比率まで近づけることを考える．

ここで，物体 i の価値を V<sub>i</sub>, 重さを W<sub>i</sub> とする．また，n個の中で(重さに関係なく)最大価値の物品の価値を V<sub>max</sub> とする．

1. K = ε V<sub>max</sub> / n というKを導入する．  
2.  V'<sub>i</sub> = floor(V<sub>i</sub> / K) とする．ここでfloor関数は小数点以下を切り捨てる関数である．
3. 各物体iが (V'<sub>i</sub>, W<sub>i</sub>) }で表される問題を修正ナップサック問題と呼ぶことにする．修正ナップサック問題での修正最適解 {X'} を求める．(X'<sub>i</sub>は 0 か 1)
4. 「修正最適解 {X'<sub>i</sub>}による価値合計」とV<sub>max</sub>とを比較し，評価値の高いほうを近似解{Xa}とする．

**計算時間**

問題になるのは手順3である．

実は，0-1ナップサック問題は，物体数 n だけではなく，最大価値V<sub>max</sub> も考慮に入れると，計算時間を O(n V<sub>max</sub>) とすることができる．このことは実際に後でプログラムを見て確認してみよう．ただし、このトリックが使えるのはナップサック問題の価値が一定刻みで量子化されていることが条件である（今回の例では整数）。

**精度**

解候補を{X}と表す(各要素が0又は1であるn次元ベクトル）．    
V({X})は解候補{X}に対する元問題での価値合計を示す．  
V'({X})は解候補{X}に対する修正問題での価値合計を示す．  
{X<sub>opt</sub>} は元問題での最適解．  
{X'} は修正問題での最適解（修正最適解)．  
{X<sub>a</sub>} は近似解．

- n K = ε V<sub>max</sub> ... 定義より
- V'<sub>i</sub> = floor(V<sub>i</sub> / K) ... 定義より
- 0 < V<sub>i</sub> - K V'<sub>i</sub> < K ... floor関数を展開
- 0 <= V({X}) - K V'({X}) < n K ... 上式は {X｝について n 個まとめても成立
- V'({X<sub>opt</sub>}) <= V'({X'}) ... 修正問題V'({X})についての最適解は{X<sub>opt</sub>}ではなくあくまで{X'}であるため
- V<sub>max</sub> <= V({X<sub>a</sub>}) ... 上記で手順4.としているため当然

これらを組み合わせていくことで，以下が導出できる．

- V({X<sub>a</sub>}) >= (1 / (1+ε)) V({X<sub>opt</sub>}) > (1-ε) V({X<sub>opt</sub>})

上式の値打ちを，自分なりの言葉で説明してみて欲しい．


# 0-1整数ナップサック問題を解くCプログラム

**アルゴリズム**

6.3.節で取り上げた0-1ナップサック問題について，価値の表現が整数に限定されるものを0-1整数ナップサック問題という．  
解を得るために必要な時間計算量の観点から言えば，0-1ナップサック問題と0-1整数ナップサック問題とで違いはない．  
しかし，整数に限定することで，解を求めるアルゴリズムには違いが表れることがある．  
ここで示すのは，整数ナップサック問題の性質を利用したアルゴリズムと，それに基づくCプログラムである．  

価値が正整数であることから，解候補はその価値がいずれも整数となる．  
ということは，価値の分布という観点から見れば，解候補の価値は整数で並べることができる．  
解候補は，仮想的に「ぴったりM万円」のナップサックを多数そろえているとイメージすると分かりやすいだろう．  
もしぴったしM万円のナップサックを作る組み合わせが幾つかあるのなら，軽い方がよい．  
もちろん，求める解は，重量制限を満たす解候補の中で，最大価値 Mmax 万円のところに存在している解候補である．  

ここまで見通せれば，あとはなんとなく想像がつくのではないだろうか．  
ある時点で，ぴったりM万円のナップサックを作ろうとするなら，それは必ずp万円のナップサックとM-p万円のナップサックの和から作ることになる．このとき，pを0万円からM万円まで変化させながら調べることになる．  
対象が複数あると，考えるべき組み合わせが整理付けにくい．  
そのため，各段階では，ある物体iについてだけ吟味する．  

以上を踏まえて，改めてアルゴリズムを考えてみよう．  
最初は物体0だけを考え，その状況下で0万円からM万円のナップサックを用意する．  
このとき，当然だが解候補が存在するのは，V<sub>0</sub>万円のナップサックだけである．  
次に，物体1を考え，その状況下で（先ほどで求めた物体0までの解候補一覧に相当する）0万円からM万円のナップサックを参照しながら，新たに(物体0から物体1までを考慮に入れた状況下での)0万円からM万円のナップサックを用意する．  
これを物体N-1まで繰り返す．  

諸君は途中から既視感を感じたはずである．これは，Floydのアルゴリズムに似ている．実際，これは動的計画法によるアルゴリズムである．Floydのアルゴリズムとの類似性を確認しておくこと．  

**Cプログラム**

Cプログラムの実装では，物体0から開始し(L81-82)，順に考慮すべき対象を増やしていく(L85のループ)．  


**計算時間**

計算時間についてこのプログラムを観察すると，O(N * v_range) になっている．  
v_rangeはこのプログラム内で現れる(修正ナップサック問題での)単品最大価値に相当する．  
プログラムに書かれているように，v_range = N \* VDashMax なので，結局 O(N<sup>2</sup> * VDashMax) と言える．

このプログラムは修正問題を扱っているので，プログラム内のVDashMaxは，元問題の変数で表すと次のようになる．

- VDashMax = floor(V<sub>max</sub> / K)

これはすなわち

- VDashMax = floor(n / ε)

であるから，結局，

- O(n<sup>2</sup> \* VDashMax) = O(n<sup>2</sup> * floor(n / ε)) = O(n<sup>3</sup> / ε)

つまり n と 1/ε の両方について多項式時間オーダーである．

**備考**

Cプログラムの実装において，与えられた問題内で考えうるナップサックの最大価値 v_range については，単品での最大価値Vmaxを用いた雑な見積もりをしている．これは計算時間の議論をするときにはそのほうが都合がよかったからであるが，無駄のないプログラムを作る観点からすると，正確な v_range のほうがありがたい．そこで，v_rangeを正確に求める方法と，それによる計算時間増について考えて見られたい．  

floydのアルゴリムに基づくプログラムでは，部分問題の解を覚える配列は１つしか用意していなかった．これは，同じ配列を書き換えても，前の部分問題の解を保持している部分を変更することがないとわかっていたからである．ここでのプログラムは，部分問題の解を覚える記憶領域 ns と nw を各部分問題ごとに用意して，大量の記憶領域を消費している．では，floydのプログラムのように，部分問題の解を覚える領域を減らすことはできるだろうか．そのようにプログラムを書き換えて，問題は発生しないだろうか．（これがすぐ正確に見通せるなら，アルゴリズムをプログラムに仕立てていく技術は一人前であろう）



In [ ]:
%%writefile Knapsack-DP-value.c
// Pseudo-polynomial Time Algorithm for Knapsack
//   for FPTAS Knapsack solution
//   kameda[at]ccs.tsukuba.ac.jp, 2021.
#include <stdio.h>
#include <stdlib.h> // calloc()

// Example-A: max value 280 = (15, 100, 90, 60, 0, 15, 0 ,0) at weight 111
// Example-B: max value  24 = () at weight 17

#define EXAMPLE_A

#ifdef EXAMPLE_A
#define WEIGHT_LIMIT 112
#define N 8 // Number of object classes, Not number of objects
struct {
	int v; // value
	int w; // weight
} obj[N] = {
	{ 15,  6},
	{100, 20},
	{ 90, 25},
	{ 60, 30},
	{ 40, 40},
	{ 15, 30},
	{ 10, 60},
	{  3, 10}
};
#endif

#ifdef EXAMPLE_B
#define WEIGHT_LIMIT 17
#define N 5 // Number of object classes, Not number of objects
struct {
	int v; // value
	int w; // weight
} obj[N] = {
	{  4,  3},
	{  5,  4},
	{ 10,  7},
	{ 11,  8},
	{ 13,  9}
};
#endif

// Pseudo-polynomial Time Algorithm for Knapsack Problem
int knapsack_byvalue(void) {
	int	VDashMax = 0; // the largest value among objects
	int v_range = 0; // value range, it should be the sum of all weights (not more than the largest value * N)
	int *ns[N]; // S: ns[i][value], i = 0 - N-1, value = 0 - N * VDashMax : object-ID to select (pick)
	int *nw[N]; // A: nw[i][value], i = 0 - N-1, value = 0 - N * VDashMax : weight of the knapsack, -1:infinite
	int max_v = 0;
	int max_w = 0;
	int i;
	int p;
	int ncalc = 0;

	// Find the largest value
	for (i = 0; i < N; i++)
		if (VDashMax < obj[i].v) VDashMax = obj[i].v;
	v_range = N * VDashMax;

	// Online memory allocation
	for (i = 0; i < N; i++) {
		if ((ns[i] = (int *)calloc(v_range, sizeof(*ns[i]))) == NULL || (nw[i] = (int *)calloc(v_range, sizeof(*ns[i]))) == NULL) {
			printf("Failed to malloc %d integers at %d.\n", v_range, i);
			return -1;
		}
	}

	// Initialization
	for (i = 0; i < N; i++) {
		for (p = 0; p <= N * VDashMax; p++) {
			ns[i][p] = -1;
			nw[i][p] = -1;
		}
	}
	ns[0][0] = -1; // nothing to select
	nw[0][0] = 0;  // weight of empty knapsack

	ns[0][obj[0].v] = 0; // Object-ID:0
	nw[0][obj[0].v] = obj[0].w; // Initial weight of object-ID:0

	// Core loop
	for (i = 1; i < N; i++) {
		// Do nothing before obj[i].v
		for (p = 0; p < obj[i].v; p++) {
			nw[i][p] = nw[i-1][p];
			ncalc++;
		}
		// Examine to make the knapsack of value=p lighter
		for (p = obj[i].v; p <= v_range; p++) {
			if ((nw[i-1][p - obj[i].v] >= 0) &&
				((nw[i-1][p] >= 0 && nw[i-1][p - obj[i].v] + obj[i].w < nw[i-1][p]) || (nw[i-1][p] < 0))) {
				nw[i][p] = nw[i-1][p - obj[i].v] + obj[i].w;
				ns[i][p] = i; // Pick Object-i here (yes/no)
			} else {
				nw[i][p] = nw[i-1][p];
			}
			ncalc++;
		}

		if (0) { // Check the array if you wish by changing 0 to 1
			int k;
			printf("ObjectID = %d\n", i);
			for (k = 0; k <= v_range; k++) printf("%3d ", k); printf("\n");
			for (k = 0; k <= v_range; k++) 	printf("%3d ", nw[i][k]); printf("\n");
			for (k = 0; k <= v_range; k++) 	printf("%3d ", ns[i][k]); printf("\n");
		}
	}

	// Find the answer
	for (p = 0; p <= v_range; p++) {
		if (max_v < p && nw[N-1][p] <= WEIGHT_LIMIT && nw[N-1][p] >= 0) {
			max_v = p;
			max_w = nw[N-1][p];
		}
	}
	printf("Answer: max_v = %d, max_w = %d\n", max_v, max_w);
	for (i = N - 1, p = max_v; i >= 0; i--) {
		printf("Object %d (total value = %3d): ", i, p);
		if (ns[i][p] >= 0) {
			printf("pick\n");
			p = p - obj[i].v;
		} else {
			printf("----\n");
			p = p; // stand-by at the same value
		}
	}
	if (0) { // To verify ns[N][] and nw[N][]
		int k,ii;
		for (k = 0; k <= v_range; k++) {
			int cc = 0;
			for (ii = 0; ii < N; ii++) {
				if (ns[ii][k] >= 0)
					cc++;
			}
			printf ("%3d ", cc);
		}
	}
	printf("Calculation = %d (Est. %d = %d * %d * %d)\n", ncalc, v_range * N, N, N, VDashMax);
	return 0;
}

// Main function
int main(int argc, char *argv[]) {
	knapsack_byvalue();
	return 0;
}


コンパイルする．デバック（学習）用にとコンパクトに書いたL105-107で親切にもWarningが出る（が意図して行っているので無視）．  
(逆に言えばこのレベルの気遣いまで今のコンパイラはしてくれている）

In [ ]:
!gcc -Wall -o Knapsack-DP-value Knapsack-DP-value.c

実行してみよう．EXAMPLE_Aだけでなく，EXAMPLE_Bでも試してみること．

In [ ]:
!./Knapsack-DP-value

時間計算量が頂点数に対して多項式時間であることを確認しておくこと．

> 余談であるが，巡回セールスマン問題に対するFPTASアルゴリズムの研究は最近でも熱心に進められている．  
 [2019年に発表されたとある研究資料](https://arxiv.org/pdf/1904.09562.pdf)中のTable 1のTextbook algorithmがここで紹介したアルゴリズムに相当する．  
 Table 1中の他のアルゴリズムに書かれている計算時間を見れば，計算量を抑え込むために研究者が大変な努力をしていることが推察できよう．


# 節末課題

1. ナップサック問題におけるFPTASアルゴリズムの精度保証  
ナップサック問題におけるFPTASアルゴリズムの精度の考察について，V({Xa}) > (1-ε) V({Xopt})を実際に導出せよ．  


2. 動的計画法アルゴリズムによる類似性  
今回の Knapsack-DP-value.c が，以前に授業で行った shortest-floyd_J.c と類似している点について，この二つが同じ動的計画法アルゴリズムであるとの観点から述べよ．

5. 気遣いの正体  
Cプログラム Knapsack-DP-value.c のコンパイルにおいて，-Wallをつけて実行すると，警告(Warning)が出る．  
一体何を心配してくれたのか説明せよ．また，今回はその心配が杞憂であることを解説せよ．

4. v_rangeの正確な見積もり  
Cプログラム Knapsack-DP-value.c の実装において，与えられた問題内で考えうるナップサックの最大価値 v_range については，単品での最大価値Vmaxを用いた雑な見積もりをしている．正確な v_rangeを求めるよう，プログラムを書き換えよ．それによる計算時間増とそれが全体の計算時間に与える影響についても考察すること．

5. 記憶領域の削減に関する検討  
floydのアルゴリムに基づくプログラムでは，部分問題の解を覚える配列は１つしか用意していなかった．これは，同じ配列を書き換えても，前の部分問題の解を保持している部分を変更することがないとわかっていたからである．Cプログラム Knapsack-DP-value.c は，部分問題の解を覚える記憶領域 ns と nw を各部分問題ごとに用意して(一つ目の添字 n を 0 から N-1まで変化できるように用意して)，大量の記憶領域を消費している．では，floydのプログラムのように，部分問題の解を覚える領域を減らすことはできるだろうか．成功すれば，nsとnwに必要な記憶領域は1/n倍になる．これが正しいかどうかをまずアルゴリズムとして検証し，正しいと見込める場合は実際にCプログラムを作成して動作を検証せよ．





# 出典

筑波大学工学システム学類  
データ構造とアルゴリズム  
担当：亀田能成  
2024/06/26 文言修正  
2022/06/21 文言修正  
2022/06/15 文言修正  
2022/05/31 文言修正  
2022/04/13 フォルダ構成を更新  
2021/06/23 初版